In [1]:
# standard
from datetime import datetime, timedelta

# installed
import aiohttp
import xml.etree.ElementTree as ElTree
import logzero
import json
# import xml.etree.ElementTree.Element as Element
import asyncio
import predcrash_connect.open_data_gouv.constants as cst

async def exec_http_request(url: str, params: dict=None, need_api: bool = False) -> str:
    """
    Execute an http request on url, using GET params, and return HTTP response.
    (This function could be used for non-SIX purpose)
    :param url: string: in six module it's always cst.SIX_URL constant
    :param params: dictionary: {"GET_param1_name": "GET_param1_value", "GET_param2_name": "GET_param2_value", ...}
    :param need_token: Must be True if the request needs a token
    :return: string containing response text, SIX should return an XML document.
    """
    if need_api:
        params['API-KEY'] = cst.API_KEY

    async with aiohttp.ClientSession() as session:
        #LOGGER.info([url, params])
        async with session.get(url, params=params) as response:
            res = await response.text()

    # If the request failed because the token was invalid, we go check our token against the one on the server.
    # LOGGER.info(res)
    return res

In [2]:
loop = asyncio.get_event_loop()
g = loop.run_until_complete(exec_http_request(cst.API_DATA_GOUV+'/datasets/carte-des-departements-2/'))

In [5]:
import json
g = json.loads(g)


{'acronym': None,
 'badges': [],
 'created_at': '2016-05-02T08:10:47.423000',
 'deleted': None,
 'description': 'Carte de chaque département extrait d\'Open Street Map, mise à jour chaque semaine.\n\n__NB__ : Les [régions](https://www.data.gouv.fr/fr/datasets/carte-des-regions-francaises-nouvelles-regions-de-2016/) sont également disponibles.\n\n\n# Contenu\n\nChaque département est livré sous forme d\'une archive ZIP qui contient plusieurs couches cartographiques au fomat shapefile (le .doc de la cartographie) :\n- places.shp : noms des villes ou des quartiers\n- roads.shp : toutes les voies de passage de l\'autoroute au chemin piéton\n- buildings.shp : l\'espace bâti\n- raillways.shp : les voies ferrées\n- waterways.shp :le réseau hydrolique\n- points.shp : une liste de point d’intérêt\n- natural.shp : zones vertes\n- landuse.shp : occupation des sol\n- admin-departement.shp : le département\n\nPar ailleurs, un fichier projet [QGis](http://www.qgis.org/fr/site/) très sommaire est fou

In [8]:
g.keys()

dict_keys(['acronym', 'badges', 'created_at', 'deleted', 'description', 'extras', 'frequency', 'frequency_date', 'id', 'last_modified', 'last_update', 'license', 'metrics', 'organization', 'owner', 'page', 'private', 'resources', 'slug', 'spatial', 'tags', 'temporal_coverage', 'title', 'uri'])

In [15]:
list_url = g['resources']
list_url

[{'checksum': None,
  'created_at': '2016-05-09T18:23:27.031000',
  'description': 'Maoytte',
  'extras': {'check:available': True,
   'check:count-availability': 82,
   'check:date': '2018-12-04T01:10:28.143000',
   'check:status': 200,
   'check:url': 'http://opendata.lexman.net/departement-976-Mayotte.zip'},
  'filesize': None,
  'filetype': 'remote',
  'format': 'shapefile',
  'id': 'dea7d37b-800e-4a66-ac6c-b09a37fec2be',
  'last_modified': '2018-12-04T02:36:01.751000',
  'latest': 'https://www.data.gouv.fr/fr/datasets/r/dea7d37b-800e-4a66-ac6c-b09a37fec2be',
  'metrics': {'nb_hits': 0,
   'nb_uniq_visitors': 0,
   'nb_visits': 0,
   'views': 42},
  'mime': 'application/zip',
  'preview_url': None,
  'published': '2016-05-09T18:23:27',
  'title': 'departement-976-Mayotte',
  'type': 'main',
  'url': 'http://opendata.lexman.net/departement-976-Mayotte.zip'},
 {'checksum': None,
  'created_at': '2016-05-09T18:23:24.060000',
  'description': None,
  'extras': {'check:available': True,

In [16]:
for i in list_url:
    print(i['extras']['check:url'])

http://opendata.lexman.net/departement-976-Mayotte.zip
http://opendata.lexman.net/departement-974-La-Reunion.zip
http://opendata.lexman.net/departement-973-Guyane.zip
http://opendata.lexman.net/departement-972-Martinique.zip
http://opendata.lexman.net/departement-971-Guadeloupe.zip
http://opendata.lexman.net/departement-2B-Haute-Corse.zip
http://opendata.lexman.net/departement-2A-Corse-du-Sud.zip
http://opendata.lexman.net/departement-95-Val-dOise.zip
http://opendata.lexman.net/departement-94-Val-de-Marne.zip
http://opendata.lexman.net/departement-93-Seine-Saint-Denis.zip
http://opendata.lexman.net/departement-92-Hauts-de-Seine.zip
http://opendata.lexman.net/departement-91-Essonne.zip
http://opendata.lexman.net/departement-90-Territoire-de-Belfort.zip
http://opendata.lexman.net/departement-89-Yonne.zip
http://opendata.lexman.net/departement-88-Vosges.zip
http://opendata.lexman.net/departement-87-Haute-Vienne.zip
http://opendata.lexman.net/departement-86-Vienne.zip
http://opendata.lexma

In [45]:
async def download_file(session: aiohttp.ClientSession, url: str):
    async with session.get(url) as response:
        assert response.status == 200
        # For large files use response.content.read(chunk_size) instead.
        g = await response.read()
        content_type = response.headers['content-type']
    with open("cardtest2.zip", "wb") as f:
        f.write(g)
    return url, content_type

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1083a04a8>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x108754b38>


In [46]:
t, v = loop.run_until_complete(download_file(aiohttp.ClientSession(), url='http://opendata.lexman.net/departement-02-Aisne.zip'))

/Users/amarchand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Creating a client session outside of coroutine is a very dangerous idea
  """Entry point for launching an IPython kernel.
Creating a client session outside of coroutine
client_session: <aiohttp.client.ClientSession object at 0x108743b00>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x108741908>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x10876f828>, 106824.822871885)]']
connector: <aiohttp.connector.TCPConnector object at 0x1087410f0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x108743b00>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1087b2cc0>, 106832.416880755)]']
connector: <aiohttp.connector.TCPConnector object at 0x108743390>


In [53]:
l = v[11:]

In [54]:
l

'/zip'

In [32]:
import re


['.', '.', '.']